In [1]:
import torch
import torch.nn as nn
import pandas as pd

In [3]:
data = pd.read_csv('sarcasm/train-balanced-sarcasm.csv')
data

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
5,0,"I don't pay attention to her, but as long as s...",only7inches,AskReddit,0,0,0,2016-09,2016-09-02 10:35:08,do you find ariana grande sexy ?
6,0,Trick or treating in general is just weird...,only7inches,AskReddit,1,-1,-1,2016-10,2016-10-23 21:43:03,What's your weird or unsettling Trick or Treat...
7,0,Blade Mastery+Masamune or GTFO!,P0k3rm4s7,FFBraveExvius,2,-1,-1,2016-10,2016-10-13 21:13:55,Probably Sephiroth. I refuse to taint his grea...
8,0,"You don't have to, you have a good build, buy ...",SoupToPots,pcmasterrace,1,-1,-1,2016-10,2016-10-27 19:11:06,What to upgrade? I have $500 to spend (mainly ...
9,0,I would love to see him at lolla.,chihawks,Lollapalooza,2,-1,-1,2016-11,2016-11-21 23:39:12,Probably count Kanye out Since the rest of his...


In [4]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split

In [5]:
train, val = train_test_split(data, test_size=0.1)

In [6]:
train.to_csv("train.csv", index=False)
val.to_csv("val.csv", index=False)

In [7]:
en = spacy.load('en_core_web_lg')
def tokenize_en(sentence):
    return [tok.text for tok in en.tokenizer(sentence)]
Q_TEXT = Field(tokenize=tokenize_en)
A_TEXT = Field(tokenize=tokenize_en)
# FR_TEXT = Field(tokenize=tokenize_fr, init_token = "<sos>", eos_token = "<eos>")

In [ ]:
data_fields = [('parent_comment', Q_TEXT), ('comment', A_TEXT)]
train,val = TabularDataset.splits(path='./', 
                                   train='train.csv', 
                                   validation='val.csv', 
                                   format='csv', 
                                   fields=data_fields)

In [17]:
data.parent_comment.iloc[5]

'do you find ariana grande sexy ?'

In [18]:
data.comment.iloc[5]

"I don't pay attention to her, but as long as she's legal I wouldn't kick her out of bed (before she took a load)"

In [6]:
tokenize_es(data.parent_comment.iloc[0])

['Yeah',
 ',',
 'I',
 'get',
 'that',
 'argument',
 '.',
 'At',
 'this',
 'point',
 ',',
 "I'd",
 'prefer',
 'is',
 'she',
 'lived',
 'in',
 'NC',
 'as',
 'well',
 '.']

In [3]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [4]:
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_len = 80):
        super().__init__()
        self.d_model = d_model
        
        # create constant 'pe' matrix with values dependant on 
        # pos and i
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = \
                math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
 
    
    def forward(self, x):
        # make embeddings relatively larger
        x = x * math.sqrt(self.d_model)
        #add constant to embedding
        seq_len = x.size(1)
        x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
        return x

In [ ]:
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']
# creates mask with 0s wherever there is padding in the input
input_msk = (input_seq != input_pad).unsqueeze(1)

In [ ]:
# create mask as before
target_seq = batch.French.transpose(0,1)
target_pad = FR_TEXT.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)
size = target_seq.size(1) # get seq_len for matrix
nopeak_mask = np.triu(np.ones(1, size, size),k=1).astype('uint8')
nopeak_mask = torch.from_numpy(nopeak_mask) == 0
target_msk = target_msk & nopeak_mask